In [ ]:
using LightGraphs, Distributions, DataFrames, ProgressMeter, RCall

In [ ]:
srand(20130810)

The decision to invest in an innovation is based on its perceived usefulness (*utility*). If the utility exceeds the barrier to adoption (measured by a threshold), the individual adopts the innovation.

The utility $u_i$ for the individual $i$ is a combination of personal benefit($p_i$), direct influence from an individuals peer group and influence from society in general. Hence,

$$
u_i(t) = \alpha_ip_i + \beta_is_i(t) + \gamma_im(t)
$$

In this equation, $s_i$ measures the impact of adoption from the peer group of an individual and $m$ measures the mainstream norm (average adoption among the entire network). Since each individual can attach differing weights to each of these three parameters, i.e., personal benefit, peer group influence and mainstream norm, we can vary the weights $\alpha_i, \beta_i, \gamma_i$ such that $\alpha_i + \beta_i + \gamma_i = 1$.

The problem description is as follows.

> Given a particular network, initial set of adopters and parameter
values $(\alpha, \beta, \gamma, p, \theta)$, we ask whether or not the innovation succeeds, that is, do most individuals
adopt the innovation. In each case, 5% of
the nodes are chosen as seed nodes at random and are set to 1, with the rest initialised at 0. The detailed structure of
the network is randomised at each set of parameter values. In order to look at the influence of
the parameters on the expected uptake, this calculation is repeated 100 times over a range of parameter
values and plotted in the $\beta$, $\gamma$ parameter space, with $0 \leq \beta \leq 1$, $0 \leq \gamma \leq 1$ and $\alpha = 1 - \beta - \gamma$. Our dependent variable is the average number of individuals adopted at the end of a predetermined set of time steps.


In [ ]:
mutable struct Network
    G::LightGraphs.SimpleGraphs.SimpleGraph{Int}
    node_status::BitVector
end

In [ ]:
function peer_influence(N::Network, node::Int)
    return sum([N.node_status[nbr] for nbr in neighbors(N.G, node)])/length(neighbors(N.G, node))
end

In [ ]:
function mainstream_norm(N::Network)
    return sum(N.node_status)/nv(N.G)
end

In [ ]:
function evolve!(N::Network, beta::Float64, gamma::Float64; p = 0.5, theta = 0.25)
    for node in shuffle(vertices(N.G))
        if N.node_status[node] == false
            s, m = peer_influence(N, node), mainstream_norm(N)
            if (beta * s + gamma * m + (1 - beta - gamma) * p) > theta
                N.node_status[node] = true
            end
        end
    end
end

## Simulation on Erdos-Renyi random graphs

In [ ]:
function simulate_er(n::Int, z::Int; T = 35, n_realizations = 100)
    parameter_space = [(beta, gamma) for beta in linspace(0, 1, 10),
                                         gamma in linspace(0, 1, 10)]
    
    output = DataFrame(r = Int[], beta = Float64[], gamma = Float64[], avg_engaged = Float64[])
    
    @showprogress 1 "Simulating..." for (beta, gamma) in parameter_space
        for r in 1:n_realizations
            g = erdos_renyi(500, 6)
            ns = falses(nv(g))
            N = Network(g, ns)

            for t in 1:T
                evolve!(N, beta, gamma)
            end
            avg_frac_engaged = sum(N.node_status)/nv(N.G)

            push!(output, [r, beta, gamma, avg_frac_engaged])
        end
    end
    
    return output
end 

In [ ]:
results = simulate_er(500, 6)

In [ ]:
describe(results_ergraph)